In [1]:
import requests
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from io import BytesIO
from urllib.parse import urlparse

In [2]:
import os, time, uuid

In [3]:
def show_image_in_cell(img_url):
    response = requests.get(img_url)
    img = Image.open(BytesIO(response.content))
    plt.figure(figsize=(20,10))
    plt.imshow(img)
    plt.show()

In [7]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials


ModuleNotFoundError: No module named 'azure'

In [5]:
TRAINING_ENDPOINT = "https://chidicustomvision.cognitiveservices.azure.com/"
training_key = "fd67989cb2284f2198c00e6ead106d28"
training_resource_id = 'subscriptions/dff1cb54-18fb-487d-ae89-d59f1ff4e7b4/resourceGroups/chidi-ndego/providers/Microsoft.CognitiveServices/accounts/chidicustomvision'

In [6]:
PREDICTION_ENDPOINT = 'https://chidicustomvision-prediction.cognitiveservices.azure.com/'
prediction_key = "4d6e1b335a5f41859f5e105d7c7d5254"
prediction_resource_id = "/subscriptions/dff1cb54-18fb-487d-ae89-d59f1ff4e7b4/resourceGroups/chidi-ndego/providers/Microsoft.CognitiveServices/accounts/chidicustomvision-Prediction"

In [7]:
training_credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(TRAINING_ENDPOINT, training_credentials)

In [8]:
trainer.api_version

'3.4-preview'

In [9]:
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(PREDICTION_ENDPOINT, prediction_credentials)

In [10]:
predictor.api_version

'3.1'

In [11]:
# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Create a new project
print ("Your Object Detection Training project has been created. Please move on.")
project_name = uuid.uuid4()
project = trainer.create_project(project_name, domain_id=obj_detection_domain.id)

Your Object Detection Training project has been created. Please move on.


In [12]:
project.as_dict()

{'id': '2c58081b-fabd-477e-955c-35de938f40f2',
 'name': 'dcbccba6-eeab-4056-86b6-22ecbc4269a6',
 'description': '',
 'settings': {'domain_id': 'da2e3a8a-40a5-4171-82f4-58522f70fbc1',
  'classification_type': 'Multilabel',
  'target_export_platforms': [],
  'use_negative_set': True,
  'image_processing_settings': {'augmentation_methods': {'rotation': True,
    'scaling': True,
    'translation': True,
    'horizontal flip': True,
    'equalize': True,
    'solarize': True,
    'padtosquare': True}}},
 'created': '2022-05-28T23:24:14.760Z',
 'last_modified': '2022-05-28T23:24:14.760Z',
 'dr_mode_enabled': False,
 'status': 'Succeeded'}

In [ ]:
project.status

'Succeeded'

: 

In [ ]:
lighter_tag = trainer.create_tag(project.id, "Lighter")

: 

#### Uploading Image with bounding box (e.g. from Cmake)

In [ ]:
# local_image_path = r"C:\Users\HP\Documents\GitHub\automated-passenger-onboarding-kiosk\object-detection\lighter_images"

: 

In [ ]:
# !dir $local_image_path

: 

In [ ]:
# # Get their bounding box coordinates

# flower_image_regions = {"image_01": [ 0.314344746162928, 0.405046480743692, 0.506493506493506, 0.34705621956618 ]}
# bird_image_regions = {"image_01": [ 0.208677685950413, 0.265161575918548, 0.19185360094451, 0.581673306772908 ]}

: 

In [ ]:
# # Upload images with tag region to the custom vision portal

# tagged_images_with_regions = []

# for file_name in flower_image_regions.keys():
#     x,y,w,h = flower_image_regions[file_name]
#     regions = [ Region(tag_id=flower_tag.id, left=x,top=y,width=w,height=h) ]
  
#     with open(os.path.join (local_image_path, file_name + ".jpg"), mode="rb") as image_contents:
#         tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), regions=regions))

# for file_name in bird_image_regions.keys():
#     x,y,w,h = bird_image_regions[file_name]
#     regions = [ Region(tag_id=bird_tag.id, left=x,top=y,width=w,height=h) ]
  
#     with open(os.path.join (local_image_path, file_name + ".jpg"), mode="rb") as image_contents:
#         tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), regions=regions))

        
# training_upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=tagged_images_with_regions))
# if not training_upload_result.is_batch_successful:
#     for image in training_upload_result.images:
#         print("Image status: ", image.status)
#     exit(-1)

: 

In [ ]:
# result of the upload

# training_upload_result.as_dict()

: 

#### Start Object Detection Training

In [ ]:
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    print ("Waiting 10 seconds...")
    time.sleep(10)

CustomVisionErrorException: Not enough images for training

: 

: 